<a href="https://colab.research.google.com/github/ekvirika/WalmartRecruiting/blob/main/notebooks/model_experiment_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install required packages
!pip install wandb torch torchvision pandas numpy matplotlib seaborn scikit-learn

# Set up Kaggle API
!pip install kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 112.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [3]:

# Upload your kaggle.json to Colab and run:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

In [4]:

# Download the dataset
!kaggle competitions download -c walmart-recruiting-store-sales-forecasting
!unzip -q walmart-recruiting-store-sales-forecasting.zip

  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 604MB/s]


In [5]:
!unzip -q train.csv.zip
!unzip -q stores.csv.zip
!unzip -q test.csv.zip
!unzip -q features.csv.zip

unzip:  cannot find or open stores.csv.zip, stores.csv.zip.zip or stores.csv.zip.ZIP.


In [6]:
!pip install dagshub mlflow --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.2/261.2 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.0/677.0 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Experiment 1

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler, OneHotEncoder
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
import xgboost as xgb
import mlflow
import mlflow.sklearn
import mlflow.xgboost
from mlflow.models.signature import infer_signature
import warnings
warnings.filterwarnings('ignore')


In [8]:
! pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 11.8 MB/s eta 0:00:00


In [9]:
# train = pd.read_csv('train.csv')
# test = pd.read_csv('test.csv')
# stores = pd.read_csv('stores.csv')
# features = pd.read_csv('features.csv')

In [10]:
# import matplotlib.pyplot as plt

# fig, axs = plt.subplots(4, 1, figsize=(15, 12), sharex=True)

# # Original Weekly Sales
# axs[0].plot(train['Weekly_Sales'], label='Weekly_Sales', color='blue')
# axs[0].set_title('Original Weekly Sales')
# axs[0].legend()

# # Rolling Average with window=4
# axs[1].plot(train['Weekly_Sales_rolling_4'], label='Rolling Mean (4 weeks)', color='orange')
# axs[1].set_title('Rolling Average - Window 4')
# axs[1].legend()

# # Rolling Average with window=13
# axs[2].plot(train['Weekly_Sales_rolling_13'], label='Rolling Mean (13 weeks)', color='green')
# axs[2].set_title('Rolling Average - Window 13')
# axs[2].legend()

# # Rolling Average with window=52
# axs[3].plot(train['Weekly_Sales_rolling_52'], label='Rolling Mean (52 weeks)', color='red')
# axs[3].set_title('Rolling Average - Window 52')
# axs[3].legend()

# plt.tight_layout()
# plt.show()


In [11]:
import mlflow
from dagshub import dagshub_logger
import os

# Set tracking URI manually
mlflow.set_tracking_uri("https://dagshub.com/ekvirika/WalmartRecruiting.mlflow")

# Use your DagsHub credentials
os.environ["MLFLOW_TRACKING_USERNAME"] = "ekvirika"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "0adb1004ddd4221395353efea2d8ead625e26197"

# Optional: set registry if you're using model registry
mlflow.set_registry_uri("https://dagshub.com/ekvirika/WalmartRecruiting.mlflow")

In [12]:
# import pandas as pd
# import numpy as np
# import mlflow
# import mlflow.sklearn
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.impute import SimpleImputer
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from xgboost import XGBRegressor
# from sklearn.metrics import mean_absolute_error
# from sklearn.model_selection import RandomizedSearchCV
# import matplotlib.pyplot as plt
# import joblib



# # ----------------------
# # Evaluation Metric (WMAE)
# # ----------------------
# def weighted_mae(y_true, y_pred, is_holiday):
#     weights = np.where(is_holiday, 5, 1)
#     return np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)

# # ----------------------
# # Data Loader
# # ----------------------
# def load_data():
#     with mlflow.start_run(run_name="XGBoost_Data_Loading", nested=True):
#         train_df = pd.read_csv('train.csv')
#         test_df = pd.read_csv('test.csv')
#         stores_df = pd.read_csv('stores.csv')
#         features_df = pd.read_csv('features.csv')

#         df = train_df.merge(features_df, on=['Store', 'Date'], how='inner') \
#                      .merge(stores_df, on='Store', how='inner')

#         mlflow.log_params({
#             "train_shape": train_df.shape,
#             "test_shape": test_df.shape,
#             "missing_values_train": train_df.isnull().sum().sum(),
#             "missing_values_test": test_df.isnull().sum().sum()
#         })

#         return df

# # ----------------------
# # Preprocessing
# # ----------------------
# def preprocess(df):
#     with mlflow.start_run(run_name="XGBoost_Cleaning", nested=True):


#       df = df.copy()

#       # Fix column naming issues
#       df["IsHoliday"] = df.pop("IsHoliday_x") if "IsHoliday_x" in df else df["IsHoliday"]
#       df.drop(columns=["IsHoliday_y"], errors='ignore', inplace=True)

#       # Filter and sort
#       df = df[df["Weekly_Sales"] > 0]
#       df["Date"] = pd.to_datetime(df["Date"])
#       df = df.sort_values(by="Date")

#       # Remove outliers
#       q_low = df["Weekly_Sales"].quantile(0.01)
#       q_high = df["Weekly_Sales"].quantile(0.99)
#       df = df[(df["Weekly_Sales"] >= q_low) & (df["Weekly_Sales"] <= q_high)]

#       # Lag features
#       for lag in [1, 2, 4, 52]:
#           df[f"lag_{lag}"] = df.groupby("Store")["Weekly_Sales"].shift(lag)

#       # Rolling statistics
#       df["rolling_mean_4"] = df.groupby("Store")["Weekly_Sales"].shift(1).rolling(window=4).mean()
#       df["rolling_std_4"] = df.groupby("Store")["Weekly_Sales"].shift(1).rolling(window=4).std()

#       # Fill missing MarkDowns
#       markdown_cols = [col for col in df.columns if "MarkDown" in col]
#       df[markdown_cols] = df[markdown_cols].fillna(0)

#       # Date features
#       df["Year"] = df["Date"].dt.year
#       df["Month"] = df["Date"].dt.month
#       df["Week"] = df["Date"].dt.isocalendar().week
#       df["DayOfWeek"] = df["Date"].dt.dayofweek
#       df["IsMonthStart"] = df["Date"].dt.is_month_start.astype(int)
#       df["IsMonthEnd"] = df["Date"].dt.is_month_end.astype(int)
#       df["Quarter"] = df["Date"].dt.quarter
#       df.drop(columns=["Date"], inplace=True)

#       mlflow.log_params({"droppped_cols": "IsHoliday_x, IsHoliday_y, Date", "fill_markdown_NaNs": "0"})

#       # Drop rows with missing lag/rolling values
#       df = df.dropna()

#       return df

# # ----------------------
# # Dynamic Pipeline Builder
# # ----------------------
# def build_pipeline(X, model=None):
#     with mlflow.start_run(run_name="XGBoost_Feature_Engineering", nested=True):

#       numeric_cols = X.select_dtypes(include=np.number).columns.tolist()
#       categorical_cols = X.select_dtypes(include=["object", "category", "bool"]).columns.tolist()

#       preprocessor = ColumnTransformer(transformers=[
#           ("num", SimpleImputer(strategy='mean'), numeric_cols),
#           ("cat", OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_cols)
#       ])

#       if model is None:
#           model = XGBRegressor(
#               n_estimators=100,
#               max_depth=6,
#               learning_rate=0.1,
#               random_state=42,
#               n_jobs=-1
#           )

#       pipeline = Pipeline([
#           ("preprocessor", preprocessor),
#           ("model", model)
#       ])

#       mlflow.log_params({"num_cols": numeric_cols, "cat_cols": categorical_cols,
#                         "simple_imputer": "mean", "cat_encoder": "one_hot", "pipeline": pipeline})

#       return pipeline

# # ----------------------
# # Runner
# # ----------------------
# def run():
#     # with mlflow.start_run(run_name="XGBoost_Training"):

#       df = load_data()
#       df = preprocess(df)

#       # Define features and target
#       X = df.drop(columns=["Weekly_Sales"])
#       y = df["Weekly_Sales"]

#       # Split
#       X_train, X_test, y_train, y_test = train_test_split(
#           X, y, test_size=0.2, random_state=42
#       )

#       # Store IsHoliday for WMAE
#       is_holiday_test = X_test["IsHoliday"].astype(bool).values if "IsHoliday" in X_test.columns else np.zeros_like(y_test)

#       # Train
#       pipeline = build_pipeline(X_train)

#       # Hyperparameter search space
#       param_dist = {
#           "model__n_estimators": [100, 200, 300],
#           "model__max_depth": [3, 6, 9],
#           "model__learning_rate": [0.05, 0.1, 0.2],
#           "model__subsample": [0.5, 0.7, 1.0],
#           "model__colsample_bytree": [0.5, 0.7, 1.0]
#       }

#       search = RandomizedSearchCV(
#           pipeline,
#           param_distributions=param_dist,
#           n_iter=20,
#           cv=3,
#           scoring="neg_mean_absolute_error",  # you could also define custom scorer using WMAE if needed
#           verbose=2,
#           n_jobs=-1,
#           random_state=42
#       )

#       search.fit(X_train, y_train)

#       best_model = search.best_estimator_
#       y_pred = best_model.predict(X_test)

#       # WMAE
#       wmae_score = weighted_mae(y_test.values, y_pred, is_holiday_test)
#       print(f"✅ WMAE: {wmae_score:.2f}")

#       mlflow.log_params(search.best_params_)
#       mlflow.log_metric("WMAE", wmae_score)
#       # mlflow.xgboost.log_model(best_model, artifact_path="xgb_model")
#       joblib.dump(best_model.named_steps["model"], "xgb_model.pkl")
#       mlflow.log_artifact("xgb_model.pkl")

#       joblib.dump(best_model, "xgb_pipeline.pkl")
#       mlflow.log_artifact("xgb_pipeline.pkl")

#       # Save feature importance plot as artifact
#       fig, ax = plt.subplots()
#       xgb_model = best_model.named_steps["model"]
#       importances = xgb_model.feature_importances_
#       print(importances)
#       ax.bar(range(len(importances)), importances)
#       plt.savefig("feature_importance.png")
#       mlflow.log_artifact("feature_importance.png")


#       return best_model

# # ----------------------
# # Entry Point
# # ----------------------
# if __name__ == "__main__":
#     try:
#         with mlflow.start_run(run_name="XGBoost_Training"):
#             model = run()
#         print("✅ Pipeline executed successfully!")
#     except Exception as e:
#         print(f"❌ Pipeline failed: {e}")


In [13]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import joblib
import warnings
warnings.filterwarnings('ignore')

# =====================================================
# BASE TRANSFORMER CLASS
# =====================================================
class BaseWalmartTransformer(BaseEstimator, TransformerMixin):
    """Base class for all Walmart transformers"""

    def __init__(self):
        self.is_fitted = False

    def fit(self, X, y=None):
        """Fit the transformer"""
        self.is_fitted = True
        return self

    def transform(self, X):
        """Transform the data"""
        if not self.is_fitted:
            raise ValueError("Transformer not fitted yet. Call fit() first.")
        return X

    def fit_transform(self, X, y=None):
        """Fit and transform in one step"""
        return self.fit(X, y).transform(X)

# =====================================================
# DATA LOADER CLASS
# =====================================================
class WalmartDataLoader:
    """Handles loading and merging of Walmart datasets"""

    def __init__(self, log_mlflow=True):
        self.log_mlflow = log_mlflow
        self.data_info = {}

    def load_data(self, train_path='train.csv', test_path='test.csv',
                  stores_path='stores.csv', features_path='features.csv'):
        """Load and merge all datasets"""

        if self.log_mlflow:
            mlflow.start_run(run_name="Data_Loading", nested=True)

        try:
            # Load datasets
            train_df = pd.read_csv(train_path)
            test_df = pd.read_csv(test_path)
            stores_df = pd.read_csv(stores_path)
            features_df = pd.read_csv(features_path)

            # Merge training data
            merged_train = train_df.merge(features_df, on=['Store', 'Date'], how='inner') \
                                  .merge(stores_df, on='Store', how='inner')

            # Store data info
            self.data_info = {
                "train_shape": train_df.shape,
                "test_shape": test_df.shape,
                "merged_train_shape": merged_train.shape,
                "missing_values_train": train_df.isnull().sum().sum(),
                "missing_values_test": test_df.isnull().sum().sum()
            }

            if self.log_mlflow:
                mlflow.log_params(self.data_info)
                mlflow.end_run()

            print(f"Data loaded successfully!")
            print(f"Training data shape: {merged_train.shape}")

            return merged_train, test_df, stores_df, features_df

        except Exception as e:
            if self.log_mlflow:
                mlflow.end_run()
            raise e

# =====================================================
# DATA CLEANING CLASS
# =====================================================
class WalmartDataCleaner(BaseWalmartTransformer):
    """Handles data cleaning and basic preprocessing"""

    def __init__(self, remove_outliers=True, outlier_quantiles=(0.01, 0.99), log_mlflow=True):
        super().__init__()
        self.remove_outliers = remove_outliers
        self.outlier_quantiles = outlier_quantiles
        self.log_mlflow = log_mlflow
        self.q_low = None
        self.q_high = None
        self.cleaning_stats = {}

    def fit(self, X, y=None):
        """Fit the cleaner - compute outlier bounds"""
        if self.log_mlflow:
            mlflow.start_run(run_name="Data_Cleaning_Fit", nested=True)

        try:
            X = X.copy()

            # Compute outlier bounds if Weekly_Sales exists
            if 'Weekly_Sales' in X.columns and self.remove_outliers:
                self.q_low = X["Weekly_Sales"].quantile(self.outlier_quantiles[0])
                self.q_high = X["Weekly_Sales"].quantile(self.outlier_quantiles[1])

                self.cleaning_stats = {
                    "outlier_lower_bound": self.q_low,
                    "outlier_upper_bound": self.q_high,
                    "original_rows": len(X)
                }

            if self.log_mlflow:
                if self.cleaning_stats:
                    mlflow.log_params(self.cleaning_stats)
                mlflow.end_run()

            return super().fit(X, y)

        except Exception as e:
            if self.log_mlflow:
                mlflow.end_run()
            raise e

    def transform(self, X):
        """Transform the data - clean and preprocess"""
        super().transform(X)  # Check if fitted

        if self.log_mlflow:
            mlflow.start_run(run_name="Data_Cleaning_Transform", nested=True)

        try:
            X = X.copy()
            original_rows = len(X)

            # Fix IsHoliday column naming issues
            if "IsHoliday_x" in X.columns:
                X["IsHoliday"] = X.pop("IsHoliday_x")
            if "IsHoliday_y" in X.columns:
                X.drop(columns=["IsHoliday_y"], inplace=True)

            # Filter positive sales (only if Weekly_Sales exists)
            if 'Weekly_Sales' in X.columns:
                X = X[X["Weekly_Sales"] > 0]

                # Remove outliers if fitted bounds exist
                if self.remove_outliers and self.q_low is not None and self.q_high is not None:
                    X = X[(X["Weekly_Sales"] >= self.q_low) & (X["Weekly_Sales"] <= self.q_high)]

            # Convert Date to datetime and sort
            if 'Date' in X.columns:
                X["Date"] = pd.to_datetime(X["Date"])
                X = X.sort_values(by="Date")

            # Fill missing MarkDown values
            markdown_cols = [col for col in X.columns if "MarkDown" in col]
            if markdown_cols:
                X[markdown_cols] = X[markdown_cols].fillna(0)

            rows_after_cleaning = len(X)

            if self.log_mlflow:
                mlflow.log_params({
                    "rows_before_cleaning": original_rows,
                    "rows_after_cleaning": rows_after_cleaning,
                    "rows_removed": original_rows - rows_after_cleaning,
                    "markdown_cols_filled": len(markdown_cols)
                })
                mlflow.end_run()

            print(f"Data cleaning completed: {original_rows} -> {rows_after_cleaning} rows")

            return X

        except Exception as e:
            if self.log_mlflow:
                mlflow.end_run()
            raise e

# =====================================================
# FEATURE ENGINEERING CLASS
# =====================================================
class WalmartFeatureEngineer(BaseWalmartTransformer):
    """Handles feature engineering - lag features, rolling stats, date features"""

    def __init__(self, lag_periods=[1, 2, 4, 52], rolling_windows=[4], log_mlflow=True):
        super().__init__()
        self.lag_periods = lag_periods
        self.rolling_windows = rolling_windows
        self.log_mlflow = log_mlflow
        self.feature_stats = {}

    def fit(self, X, y=None):
        """Fit feature engineer - just store parameters"""
        if self.log_mlflow:
            mlflow.start_run(run_name="Feature_Engineering_Fit", nested=True)

        try:
            self.feature_stats = {
                "lag_periods": self.lag_periods,
                "rolling_windows": self.rolling_windows,
                "has_weekly_sales": 'Weekly_Sales' in X.columns,
                "has_date": 'Date' in X.columns,
                "has_store": 'Store' in X.columns
            }

            if self.log_mlflow:
                mlflow.log_params(self.feature_stats)
                mlflow.end_run()

            return super().fit(X, y)

        except Exception as e:
            if self.log_mlflow:
                mlflow.end_run()
            raise e

    def transform(self, X):
        """Transform data - create features"""
        super().transform(X)

        if self.log_mlflow:
            mlflow.start_run(run_name="Feature_Engineering_Transform", nested=True)

        try:
            X = X.copy()
            original_features = len(X.columns)

            # Create lag features
            if 'Weekly_Sales' in X.columns and 'Store' in X.columns:
                for lag in self.lag_periods:
                    X[f"lag_{lag}"] = X.groupby("Store")["Weekly_Sales"].shift(lag)

                # Create rolling statistics
                for window in self.rolling_windows:
                    X[f"rolling_mean_{window}"] = X.groupby("Store")["Weekly_Sales"].shift(1).rolling(window=window).mean()
                    X[f"rolling_std_{window}"] = X.groupby("Store")["Weekly_Sales"].shift(1).rolling(window=window).std()

            # Create date features
            if 'Date' in X.columns:
                X["Year"] = X["Date"].dt.year
                X["Month"] = X["Date"].dt.month
                X["Week"] = X["Date"].dt.isocalendar().week
                X["DayOfWeek"] = X["Date"].dt.dayofweek
                X["IsMonthStart"] = X["Date"].dt.is_month_start.astype(int)
                X["IsMonthEnd"] = X["Date"].dt.is_month_end.astype(int)
                X["Quarter"] = X["Date"].dt.quarter

                # Drop Date column after feature extraction
                X.drop(columns=["Date"], inplace=True)

            # Drop rows with NaN values (from lag/rolling features)
            rows_before_dropna = len(X)
            X = X.dropna()
            rows_after_dropna = len(X)

            new_features = len(X.columns)

            if self.log_mlflow:
                mlflow.log_params({
                    "original_features": original_features,
                    "new_features": new_features,
                    "features_added": new_features - original_features,
                    "rows_before_dropna": rows_before_dropna,
                    "rows_after_dropna": rows_after_dropna,
                    "rows_dropped_due_to_nan": rows_before_dropna - rows_after_dropna
                })
                mlflow.end_run()

            print(f"Feature engineering completed: {original_features} -> {new_features} features")
            print(f"Rows after removing NaNs: {rows_before_dropna} -> {rows_after_dropna}")

            return X

        except Exception as e:
            if self.log_mlflow:
                mlflow.end_run()
            raise e

# =====================================================
# MODEL PIPELINE CLASS
# =====================================================
class WalmartModelPipeline(BaseWalmartTransformer):
    """Handles model preprocessing and training"""

    def __init__(self, model=None, log_mlflow=True):
        super().__init__()
        self.model = model
        self.log_mlflow = log_mlflow
        self.preprocessor = None
        self.pipeline = None
        self.numeric_cols = None
        self.categorical_cols = None

    def fit(self, X, y=None):
        """Fit the preprocessing pipeline"""
        if self.log_mlflow:
            mlflow.start_run(run_name="Model_Pipeline_Fit", nested=True)

        try:
            # Identify column types
            self.numeric_cols = X.select_dtypes(include=np.number).columns.tolist()
            self.categorical_cols = X.select_dtypes(include=["object", "category", "bool"]).columns.tolist()

            # Create preprocessor
            transformers = []
            if self.numeric_cols:
                transformers.append(("num", SimpleImputer(strategy='mean'), self.numeric_cols))
            if self.categorical_cols:
                transformers.append(("cat", OneHotEncoder(drop='first', handle_unknown='ignore'), self.categorical_cols))

            self.preprocessor = ColumnTransformer(transformers=transformers)

            # Create default model if none provided
            if self.model is None:
                self.model = XGBRegressor(
                    n_estimators=100,
                    max_depth=6,
                    learning_rate=0.1,
                    random_state=42,
                    n_jobs=-1
                )

            # Create pipeline
            self.pipeline = Pipeline([
                ("preprocessor", self.preprocessor),
                ("model", self.model)
            ])

            if self.log_mlflow:
                mlflow.log_params({
                    "numeric_cols_count": len(self.numeric_cols),
                    "categorical_cols_count": len(self.categorical_cols),
                    "numeric_cols": self.numeric_cols[:10],  # Log first 10 to avoid too long
                    "categorical_cols": self.categorical_cols,
                    "model_type": type(self.model).__name__
                })
                mlflow.end_run()

            return super().fit(X, y)

        except Exception as e:
            if self.log_mlflow:
                mlflow.end_run()
            raise e

    def transform(self, X):
        """Transform using the fitted pipeline preprocessor"""
        super().transform(X)

        if self.pipeline is None:
            raise ValueError("Pipeline not fitted yet.")

        # Only transform using preprocessor, not the full pipeline
        return self.pipeline.named_steps['preprocessor'].transform(X)

    def get_pipeline(self):
        """Get the full pipeline for training"""
        if not self.is_fitted:
            raise ValueError("Pipeline not fitted yet.")
        return self.pipeline

# =====================================================
# EVALUATION METRICS CLASS
# =====================================================
class WalmartEvaluator:
    """Handles model evaluation with WMAE and other metrics"""

    def __init__(self):
        pass

    def weighted_mae(self, y_true, y_pred, is_holiday):
        """Calculate Weighted Mean Absolute Error"""
        weights = np.where(is_holiday, 5, 1)
        return np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)

    def evaluate_model(self, model, X_test, y_test, is_holiday_test=None):
        """Evaluate model with multiple metrics"""
        y_pred = model.predict(X_test)

        # Standard MAE
        mae_score = mean_absolute_error(y_test, y_pred)

        # WMAE if holiday info available
        wmae_score = None
        if is_holiday_test is not None:
            wmae_score = self.weighted_mae(y_test, y_pred, is_holiday_test)

        metrics = {
            'mae': mae_score,
            'wmae': wmae_score
        }

        return metrics, y_pred

# =====================================================
# MAIN PIPELINE CLASS
# =====================================================
class WalmartSalesPipeline:
    """Main pipeline orchestrating all components"""

    def __init__(self, log_mlflow=True, experiment_name="Walmart_Sales_OOP"):
        self.log_mlflow = log_mlflow
        self.experiment_name = experiment_name

        # Initialize components
        self.data_loader = WalmartDataLoader(log_mlflow=log_mlflow)
        self.data_cleaner = WalmartDataCleaner(log_mlflow=log_mlflow)
        self.feature_engineer = WalmartFeatureEngineer(log_mlflow=log_mlflow)
        self.model_pipeline = WalmartModelPipeline(log_mlflow=log_mlflow)
        self.evaluator = WalmartEvaluator()

        # Pipeline state
        self.is_fitted = False
        self.best_model = None
        self.best_params = None
        self.training_data = None

        if self.log_mlflow:
            mlflow.set_experiment(self.experiment_name)

    def fit(self, train_path='train.csv', test_path='test.csv',
            stores_path='stores.csv', features_path='features.csv',
            hyperparameter_search=True, cv_folds=3, n_iter=20):
        """Fit the entire pipeline"""

        if self.log_mlflow:
            mlflow.start_run(run_name="Walmart_Pipeline_Training")

        try:
            print("="*60)
            print("WALMART SALES PIPELINE - TRAINING")
            print("="*60)

            # 1. Load data
            print("\n1. Loading data...")
            train_data, test_data, stores_data, features_data = self.data_loader.load_data(
                train_path, test_path, stores_path, features_path
            )

            # 2. Clean data
            print("\n2. Cleaning data...")
            self.data_cleaner.fit(train_data)
            train_cleaned = self.data_cleaner.transform(train_data)

            # 3. Engineer features
            print("\n3. Engineering features...")
            self.feature_engineer.fit(train_cleaned)
            train_processed = self.feature_engineer.transform(train_cleaned)

            # Store processed data
            self.training_data = train_processed.copy()

            # 4. Prepare features and target
            print("\n4. Preparing features and target...")
            X = train_processed.drop(columns=["Weekly_Sales"])
            y = train_processed["Weekly_Sales"]

            # 5. Split data
            print("\n5. Splitting data...")
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=0.2, random_state=42
            )

            # Extract IsHoliday for WMAE
            is_holiday_test = X_test["IsHoliday"].astype(bool).values if "IsHoliday" in X_test.columns else np.zeros_like(y_test, dtype=bool)

            # 6. Fit model pipeline
            print("\n6. Fitting model pipeline...")
            self.model_pipeline.fit(X_train, y_train)
            pipeline = self.model_pipeline.get_pipeline()

            # 7. Hyperparameter tuning
            if hyperparameter_search:
                print("\n7. Hyperparameter tuning...")
                param_dist = {
                    "model__n_estimators": [100, 200, 300],
                    "model__max_depth": [3, 6, 9],
                    "model__learning_rate": [0.05, 0.1, 0.2],
                    "model__subsample": [0.5, 0.7, 1.0],
                    "model__colsample_bytree": [0.5, 0.7, 1.0]
                }

                search = RandomizedSearchCV(
                    pipeline,
                    param_distributions=param_dist,
                    n_iter=n_iter,
                    cv=cv_folds,
                    scoring="neg_mean_absolute_error",
                    verbose=1,
                    n_jobs=-1,
                    random_state=42
                )

                search.fit(X_train, y_train)
                self.best_model = search.best_estimator_
                self.best_params = search.best_params_

                if self.log_mlflow:
                    mlflow.log_params(self.best_params)

            else:
                print("\n7. Training model without hyperparameter search...")
                pipeline.fit(X_train, y_train)
                self.best_model = pipeline

            # 8. Evaluate model
            print("\n8. Evaluating model...")
            metrics, y_pred = self.evaluator.evaluate_model(
                self.best_model, X_test, y_test, is_holiday_test
            )

            print(f"✅ MAE: {metrics['mae']:.2f}")
            if metrics['wmae'] is not None:
                print(f"✅ WMAE: {metrics['wmae']:.2f}")

            if self.log_mlflow:
                mlflow.log_metrics(metrics)

            # 9. Save artifacts
            print("\n9. Saving model artifacts...")
            self._save_artifacts()

            self.is_fitted = True

            print("\n" + "="*60)
            print("✅ PIPELINE TRAINING COMPLETED!")
            print("="*60)

            if self.log_mlflow:
                mlflow.end_run()

            return self.best_model, metrics

        except Exception as e:
            if self.log_mlflow:
                mlflow.end_run()
            raise e

    def predict(self, new_data):
        """Make predictions on new data"""
        if not self.is_fitted:
            raise ValueError("Pipeline not fitted yet. Call fit() first.")

        # Apply same preprocessing steps
        cleaned_data = self.data_cleaner.transform(new_data)
        processed_data = self.feature_engineer.transform(cleaned_data)

        # Remove target column if it exists
        if 'Weekly_Sales' in processed_data.columns:
            processed_data = processed_data.drop(columns=['Weekly_Sales'])

        # Make predictions
        predictions = self.best_model.predict(processed_data)

        return predictions

    def _save_artifacts(self):
        """Save model artifacts"""
        try:
            # Save pipeline
            joblib.dump(self.best_model, "walmart_pipeline.pkl")
            if self.log_mlflow:
                mlflow.log_artifact("walmart_pipeline.pkl")

            # Save individual components
            joblib.dump(self.data_cleaner, "data_cleaner.pkl")
            joblib.dump(self.feature_engineer, "feature_engineer.pkl")
            joblib.dump(self.model_pipeline, "model_pipeline.pkl")

            if self.log_mlflow:
                mlflow.log_artifact("data_cleaner.pkl")
                mlflow.log_artifact("feature_engineer.pkl")
                mlflow.log_artifact("model_pipeline.pkl")

            # Feature importance plot
            if hasattr(self.best_model.named_steps["model"], 'feature_importances_'):
                fig, ax = plt.subplots(figsize=(10, 6))
                importances = self.best_model.named_steps["model"].feature_importances_
                indices = np.argsort(importances)[::-1][:20]  # Top 20 features

                ax.bar(range(len(indices)), importances[indices])
                ax.set_title("Top 20 Feature Importances")
                ax.set_xlabel("Feature Index")
                ax.set_ylabel("Importance")

                plt.tight_layout()
                plt.savefig("feature_importance.png")

                if self.log_mlflow:
                    mlflow.log_artifact("feature_importance.png")

                plt.close()

        except Exception as e:
            print(f"Warning: Could not save some artifacts: {e}")

# =====================================================
# USAGE EXAMPLE
# =====================================================
if __name__ == "__main__":
    # Initialize and run pipeline
    pipeline = WalmartSalesPipeline(
        log_mlflow=True,
        experiment_name="Walmart_Sales_OOP_Pipeline"
    )

    try:
        # Train the pipeline
        best_model, metrics = pipeline.fit(
            hyperparameter_search=True,
            cv_folds=3,
            n_iter=10  # Reduced for faster testing
        )

        print(f"\n🎉 Training completed successfully!")
        print(f"Best parameters: {pipeline.best_params}")

    except Exception as e:
        print(f"❌ Pipeline failed: {e}")
        import traceback
        traceback.print_exc()

2025/07/31 11:54:50 INFO mlflow.tracking.fluent: Experiment with name 'Walmart_Sales_OOP_Pipeline' does not exist. Creating a new experiment.


WALMART SALES PIPELINE - TRAINING

1. Loading data...
🏃 View run Data_Loading at: https://dagshub.com/ekvirika/WalmartRecruiting.mlflow/#/experiments/1/runs/7511b1d63cc94b24853fd6bd5a505361
🧪 View experiment at: https://dagshub.com/ekvirika/WalmartRecruiting.mlflow/#/experiments/1
Data loaded successfully!
Training data shape: (421570, 17)

2. Cleaning data...
🏃 View run Data_Cleaning_Fit at: https://dagshub.com/ekvirika/WalmartRecruiting.mlflow/#/experiments/1/runs/1996f0510a8340b1a7fb4ede7c8e232b
🧪 View experiment at: https://dagshub.com/ekvirika/WalmartRecruiting.mlflow/#/experiments/1
🏃 View run Data_Cleaning_Transform at: https://dagshub.com/ekvirika/WalmartRecruiting.mlflow/#/experiments/1/runs/121a63d25d704556a76a869fb86ed17b
🧪 View experiment at: https://dagshub.com/ekvirika/WalmartRecruiting.mlflow/#/experiments/1
Data cleaning completed: 421570 -> 413212 rows

3. Engineering features...
🏃 View run Feature_Engineering_Fit at: https://dagshub.com/ekvirika/WalmartRecruiting.mlfl

In [14]:
model_uri = "runs:/8008f1cad2b74f29bc2d836bb5699af7/model_pipeline.pkl"


In [2]:
!pip install mlflow --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 680.6/680.6 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 13.1 MB/s eta 0:00:00


In [6]:
import mlflow
import joblib
from mlflow.models import ModelSignature, infer_signature

# Load the model.pkl manually
model = joblib.load("mlruns/8008f1cad2b74f29bc2d836bb5699af7/artifacts/model_pipeline.pkl")  # or from a URL

# Optional: infer signature (if you have data)
# signature = infer_signature(X_test, model.predict(X_test))

# Start a new run and log model properly for registration
with mlflow.start_run() as run:
    mlflow.sklearn.log_model(model, artifact_path="model")  # or xgboost.log_model


FileNotFoundError: [Errno 2] No such file or directory: 'mlflow-artifacts:/e0cbe91d54c64e1d839ca8a3c7668be3/8008f1cad2b74f29bc2d836bb5699af7/artifacts/model_pipeline.pkl'

In [9]:
import mlflow
import joblib

# This assumes your model.pkl is inside the 'model' artifact folder
model_uri = "mlflow-artifacts:/e0cbe91d54c64e1d839ca8a3c7668be3/8008f1cad2b74f29bc2d836bb5699af7/artifacts/model_pipeline.pkl"

# Download the file locally (MLflow handles remote storage)
local_path = mlflow.artifacts.download_artifacts(model_uri)
model = joblib.load(local_path)

# Log again for registration
with mlflow.start_run() as run:
    mlflow.sklearn.log_model(model, artifact_path="xgb_model")  # or xgboost.log_model if it's XGBoost
    print("Re-logged for registration. Run ID:", run.info.run_id)


MlflowException: When an mlflow-artifacts URI was supplied, the tracking URI must be a valid http or https URI, but it was currently set to file:///content/mlruns. Perhaps you forgot to set the tracking URI to the running MLflow server. To set the tracking URI, use either of the following methods:
1. Set the MLFLOW_TRACKING_URI environment variable to the desired tracking URI. `export MLFLOW_TRACKING_URI=http://localhost:5000`
2. Set the tracking URI programmatically by calling `mlflow.set_tracking_uri`. `mlflow.set_tracking_uri('http://localhost:5000')`

In [5]:
import mlflow
import joblib
# from my_module import WalmartModelPipeline  # <- Make sure this line works


mlflow.set_tracking_uri("https://dagshub.com/ekvirika/WalmartRecruiting.mlflow")

# Set your run ID and artifact path
run_id = "8008f1cad2b74f29bc2d836bb5699af7"
artifact_path = "model_pipeline.pkl"

# Download the file to a temporary location
local_path = mlflow.artifacts.download_artifacts(
    run_id=run_id,
    artifact_path=artifact_path
)

# Load the model from the downloaded path
model = joblib.load(local_path)

# # Log again as an MLflow model
with mlflow.start_run() as run:
    mlflow.sklearn.log_model(model, artifact_path="xgb_model")
    print("Model logged at:", f"runs:/{run.info.run_id}/xgb_model")


AttributeError: Can't get attribute 'WalmartModelPipeline' on <module '__main__'>

In [7]:
import joblib
import mlflow

# Optional: use cloudpickle if custom classes are involved
model = joblib.load("mlflow-artifacts:/e0cbe91d54c64e1d839ca8a3c7668be3/8008f1cad2b74f29bc2d836bb5699af7/artifacts/model_pipeline.pkl")


FileNotFoundError: [Errno 2] No such file or directory: 'mlflow-artifacts:/e0cbe91d54c64e1d839ca8a3c7668be3/8008f1cad2b74f29bc2d836bb5699af7/artifacts/model_pipeline.pkl'